In [ ]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, subprocess
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator   
import nltk
from nltk.corpus import stopwords  
from nltk.stem import PorterStemmer
ps = PorterStemmer()
sw=stopwords.words('english') 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')
#df, code = generate_code_from_db(ctx, qgroup=4028)

In [ ]:
def norm(s):
    s=re.sub('\s','',s)
    return s 
sheet_name='Scan Metrics'
df_template = pd.read_excel(r"C:\Users\timko\Downloads\BOD2301Templates.xlsx", header=0, sheet_name=sheet_name,  usecols='B:ZZ', dtype=str)   
sheet_name=sheet_name.replace(' ', '')
cols = list(df_template.columns)  
fields = list(df_template.columns)  
fields = list(map( lambda s:  s.replace('%','Per'), fields)) 
fields = list(map( normalize_var, fields)) 
fields = fields + ['FK_HOST_SCAN']
s,p,c='','',''
for field in fields:   
    s += f'\t, {field} \n' 
    if 'Per' in field:
        p += f'\t, @{field} DECIMAL(10,2) = NULL \n'  
    elif 'FK_' in field:
        p += f'\t, @{field} NVARCHAR(100) = NULL \n'  
    else:
        p += f'\t, @{field} INT = NULL \n' 
 
c = re.sub('=|@','',p) 
sql=''
with open(r"C:\Users\timko\Desktop\BOD2301_CDMDATA_CRUD.sql", 'r' , encoding="utf8", errors='replace') as f:
    sql=f.read()
sql=sql.replace('--@fields', p)
sql=sql.replace('--fields', s)
sql=sql.replace('--cols', c) 
sql=sql.replace('BOD2301_CDMDATA', 'BOD2301_CDM_'+sheet_name)



In [ ]:

with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\datacall\BOD2301_CDM_'+sheet_name+'_CRUD.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{sql}' ) 

In [ ]:
for c in df_template.columns:
    f = c.replace('%','Per') 
    f = normalize_var(f)
    print( f'\t\t<CB:DataField  DBColumnName="{f}" Require="False" ImportColumnName="{c}" runat="server"/>  ')

In [ ]:
df=sql_todf("  SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME IN( 'vwCDMHostScan')  ", connstr)   
 
for i,r in df.iterrows():
    c=r['COLUMN_NAME']
    col = """  , c AS [c]""".replace('c', c)
    print(f' {col} ')

In [ ]:
df=sql_todf("  SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME IN(  'BOD2301_CDM_ScanMetrics')  ", connstr)   
 
cols = df[2:]['COLUMN_NAME'].to_list()
for f in cols:
    col = """  , {  field: "1", title: "2" }"""
    col=col.replace('1',f)
    col=col.replace('2',to_proper(f).replace('Percentage', ' pct.'))
    print(col)
    # print(f', {f} AS [{to_proper(f)}]')

In [ ]:
df=sql_todf("  SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'BOD2301_CDM_HostMetrics'  ", connstr)   
 
cols = df[2:]['COLUMN_NAME'].to_list()
for f in cols:
    col = """  , {  field: "1", title: "2" }"""
    col=col.replace('1',f)
    col=col.replace('2',to_proper(f).replace('Percentage', ' pct.'))
    print(col)
    # print(f', {f} AS [{to_proper(f)}]')

In [ ]:
ag = set(sql_todf("  SELECT Acronym FROM [Component List] WHERE IsActive=1 AND Component LIKE '%Administration%' AND FK_PK_Component IS NULL  ", connstr)['Acronym']) 
ag = list(ag)
ag

In [ ]:
df=sql_todf("  SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'BOD2301_CDM_ScanMetrics'  ", connstr)   
lod = []
cols = df[2:31]['COLUMN_NAME'].to_list()
for i in range(len(ag)):
    d={}
    for c in cols:
        d[c]=random.randint(50, 100)
        if 'Agency' in c:
            d[c]=ag[i] + ''
        if 'FK_HOST_SCAN' in c:
            d[c]= 'FK_HOST_SCAN_1'
            
    lod.append(d)
lod
df = pd.DataFrame(lod) # .to_csv(r"C:\Users\timko\Downloads\BOD2301_CDM_ScanMetrics.csv", index=False)
df.to_excel(rf'C:\Users\timko\Downloads\BOD2301_CDM_ScanMetrics.xlsx',  sheet_name='Scan Metrics', index=False) 
df

In [ ]:
ntb = '<telerik:RadNumericTextBox ID="field" runat="server" MinValue="0"/>'
tb = '<asp:TextBox runat="server" ID="field" Value="" />'
lbl = '<asp:Label runat="server" ID="field" Text="" />'
frm , s = "<tr> <td>{A}</td> <td>{B}</td> <td>{C}</td> <td>{D}</td> <td>{E}</td> <td>{F}</td> <td>{G}</td> <td>{H}</td>  </tr>", ''
 

In [ ]:
s, ss, u, uu = '','','',''
for i, r in df.iterrows():
    c = f'{normalize_var(r["COLUMN_NAME"])}'  
    field= r["COLUMN_NAME"]
    u += f", CASE WHEN {field} < 0 THEN 'DNS' ELSE {field} END AS {field}_DISPLAY -- DNS (DID NOT RESPOND)\n"
    if re.match('.*App.*|.*CDMP.*', c):
        s +=  f", {field} " # \n" 
        ss +=  f", {i}" # f", @{field} \n" 
    if 'DATE' in r["DATA_TYPE"]:
        uu += f", FORMAT({field}, 'MM/dd/yyyy') AS {field} \n" 
    else:
        uu += f', {field} \n'
print(s)
print(ss) 
 

In [ ]:
for i,r in df.iterrows():
    c = f'{normalize_var(r["COLUMN_NAME"])}' 
    dfld = f'<CB:DataField  DBColumnName="{c}" Require="False" ImportColumnName="{c}" runat="server"/>  '
    if 'Perc' in c:
        dfld = f'<CB:DataField  DBColumnName="{c}" Require="False" DataType="System.DateTime" NonDatePlaceholder="01/01/1900" ImportColumnName="{c}"  runat="server"/>  '
    # print(s) 
    print(dfld)

In [ ]:


for c in df.columns:
    n = f'{normalize_var(c)}' 
    s = f',{n} NVARCHAR(MAX) NULL ' 
    # print(s)
    dfld = f'<CB:DataField  DBColumnName="{n}" Require="False" ImportColumnName="{c}" runat="server"/>  '
    if c[-4:] == "Date":
        dfld = f'<CB:DataField  DBColumnName="{n}" Require="False" DataType="System.DateTime" NonDatePlaceholder="01/01/1900" ImportColumnName="{c}"  runat="server"/>  '
    print(dfld )

In [15]:

df = pd.read_excel(r'C:\Users\timko\Downloads\BOD 23-01 - Calculations and Data Sources - 061225.xlsx', sheet_name='Custom Query', header=0, usecols='A:Z' )  
df.columns=[c.replace('\n' ,' ') for c in df.columns] 
for c in df.columns:
    print(f", '' AS [{c}] ")

, '' AS [Agency Name] 
, '' AS [Agency] 
, '' AS [Submission Date] 
, '' AS [CDM  Assets Discovered] 
, '' AS [Appended  Assets Discovered] 
, '' AS [CDM  Vulnerability Enumeration] 
, '' AS [Appended  Vulnerability Enumeration] 
, '' AS [CDM Credentialed Scan] 
, '' AS [Appended Credentialed Scan] 
, '' AS [CDM Endpoints] 
, '' AS [Appended Endpoints] 
, '' AS [CDM Networking Devices] 
, '' AS [Appended Networking Devices] 
, '' AS [CDM Mobile Devices] 
, '' AS [Appended Mobile Devices] 
, '' AS [CDM Unknown] 
, '' AS [Appended Unknown] 
, '' AS [CDM Other IO Devices] 
, '' AS [Appended Other IO Devices] 
, '' AS [CDM Vulnerability Signatures] 
, '' AS [Appended Vulnerability Signatures] 


In [ ]:
df = pd.read_excel(r'C:\Users\Tim\Downloads\fedramp.xlsx', sheet_name='Agency - CSP Table', header=1, usecols='B:Z' )  
df.columns=[c.replace(' ' ,'') for c in df.columns] 
df
# df.to_excel(r'C:\Users\Tim\Downloads\fedramp_admin_import.xlsx', sheet_name='Admin Import', index=False)


In [ ]:
df = pd.read_excel(r'C:\Users\timko\Downloads\TLP RED - FY25 HVA List - CyberScope Upload.xlsx', sheet_name='Export', header=0, usecols='A:Z' )   
df['PK_HVA'] = df['HVA_ID'].apply(id)
df['TierPK'] = df['Tier'].apply(tier)
df

In [ ]:

up = "UPDATE fsma_HVAs SET Tier='t', AgencyRank='ar' WHERE PK_HVA = 'pk' ;"
sql=''
for i, row in df[:].iterrows():
    s=up.replace("'t'", row["TierPK"])
    s=s.replace("'pk'", row["PK_HVA"])
    s=s.replace("'ar'", str(row["Rank within FHE"]))
    
    sql=sql+s+'\n'
print( sql )


with open(r'y:\tkopp\tscript.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{sql}' ) 



In [ ]:
fields = pd.read_csv(f'{ctx.get_dest()}fedramp.csv') 
df=normalizedf(fields)
df

In [ ]:
ag = set(sql_todf("  SELECT Component FROM [Component List] WHERE IsActive=1 AND FK_PK_Component IS NULL  ", connstr)['Component']) 
ag = set(ag)
ag

In [ ]:
for i, row in df[:].iterrows():
    s=f",@{row['COLUMN_NAME']} {row['DATA_TYPE']}"
    print(s)

In [ ]:
def df_fromTemplateColumn(path): 
    lst=[]
    cols=[]
    with open(path, 'r') as f:  
        g=re.search("<Columns>(.*)<\/Columns", f.read() , flags=re.DOTALL)   
        lst=g.groups(0)[0].split('<CB:CBGridTemplateColumn')[1:] 
        for l in lst: 
            un = re.search('UniqueName="(\w+)"\s', l ).groups(0)[0] 
            ht = un
            if 'HeaderText' in l:
                try: ht = re.search('HeaderText="([A-Za-z\s]*)"', l ).groups(0)[0]  
                except:  pass 
            cols.append({  'UniqueName': un ,'HeaderText':ht   })
            
    return pd.DataFrame(cols)
df = df_fromTemplateColumn(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\BODSCuBA\2024\2024_A_SCuBATenantInventory_1.aspx')
df

In [ ]:

for i,r in df.iterrows(): 
    un = r['UniqueName'] 
    ht = r['HeaderText'] 
    s = f'<CB:DataField  DBColumnName="{un.replace(" ","")}" Require="False" ImportColumnName="{ht}" runat="server"/>  '
    if "Date" in un: 
        s = f'<CB:DataField  DBColumnName="{un.replace(" ","")}" Require="False" ImportColumnName="{ht}"  runat="server"/>  '
    print(s)

In [ ]:
for i, r in df.iterrows(): 
    DATA_TYPE=r["DATA_TYPE"]
    COLUMN_NAME=r["COLUMN_NAME"]
    HEADER=r["HEADER"]
    PK_PICKLISTTYPE=str(r["PK_PICKLISTTYPE"] )
    s=f'<CB:DataField DBColumnName="{COLUMN_NAME}" ImportColumnName="{HEADER}" SprocParamName="{COLUMN_NAME}" runat="server"/>' 
    s=f',{COLUMN_NAME} '
    if PK_PICKLISTTYPE != '0': s=s.replace('runat',' PickListTypeID="'+ PK_PICKLISTTYPE +'" runat')   
    
    e=f',{COLUMN_NAME} AS [{HEADER}] '
    if PK_PICKLISTTYPE != '0': e=f', (SELECT DisplayValue FROM vwPICKLISTS WHERE PK_Picklist={COLUMN_NAME}) AS [{HEADER}]' 
    if 'Date' in COLUMN_NAME:
        e=f" , CASE WHEN {COLUMN_NAME} LIKE '%2000%' THEN 'NA' ELSE FORMAT({COLUMN_NAME}, 'MM/dd/yyyy') END AS [{HEADER}]"
    print(s)

In [ ]:

for i, row in df[:].iterrows():
    with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
        s=f.read()  
    DATA_TYPE=row['DATA_TYPE']
    COLUMN_NAME=row['COLUMN_NAME']
    HEADER=row['HEADER']
    PK_PICKLISTTYPE=row['PK_PICKLISTTYPE']
    s=s.replace('{COLUMN_NAME}', COLUMN_NAME)
    if 'INT' in DATA_TYPE:  
        s=s.replace('{Bind_COLUMN_NAME}', COLUMN_NAME+'_Display')
        s=s.replace('<%-- --%>', f'<telerik:RadDropDownList ID="{COLUMN_NAME}" DataValueField="YN" runat="server" />')  
    else:
        s=s.replace('{Bind_COLUMN_NAME}', COLUMN_NAME )
        if 'Date' in COLUMN_NAME:  
            # print("const "+COLUMN_NAME+" = new CBDatePicker({ id: $(`[id$=_"+COLUMN_NAME+"]`).prop('id'), allowNa: true  }); ")  
            s=s.replace('<%-- --%>', f'<asp:HiddenField ID="{COLUMN_NAME}" runat="server" />') 
        else: 
            s=s.replace('<%-- --%>', f'<telerik:RadTextBox ID="{COLUMN_NAME}" runat="server" />') 
    #print(f"[{COLUMN_NAME}] [NVARCHAR](4000) NULL,")  
    #print('{ field: "'+COLUMN_NAME+'", title: "'+HEADER+'" },')
    print (s)

In [ ]:
 
def get_gridcols(df,out=False):
    ldf=df.to_dict(orient='records') 
    code,ddl,dv,sel,dtf=[],[],[],[],[]
    with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
        t=f.read() 
        
        for d in ldf: 
            col=d['COLUMN_NAME'] 
            s=t.replace("{COLUMN_NAME}",d['COLUMN_NAME'])  
            if 'INT' in d['DATA_TYPE'].upper():
                s=s.replace('{Bind_COLUMN_NAME}',f"{d['COLUMN_NAME']}_Display")
                s=s.replace('<%-- --%>', f'<telerik:RadDropDownList ID="{col}" DataValueField="YN" runat="server" />')
                ddl.append(f'BindDDL(e, "{col}", "{col}", "{col}")')
                sel.append(f',(SELECT DisplayValue FROM vwPICKLISTS WHERE PK_Picklist={col}) AS {col}') 
            else: 
                s=s.replace('{Bind_COLUMN_NAME}',f"{d['COLUMN_NAME']}")
                s=s.replace('<%-- --%>', f'<telerik:RadTextBox ID="{col}" runat="server" />')
                dv.append(f'DirectCast(e.Item.FindControl($"{col}"), RadTextBox).Text = drVal("{col}").ToString()')
                sel.append(f',{col}') 
            code.append(f'{s}')     
    code='\n'.join(code) 
    ddl='\n'.join(ddl)
    dv= '\n'.join(dv)
    sel= '\n'.join(sel)[1:] 
    return {'code':code, 'ddl':ddl,'dv':dv,'sel':sel, 'dtf': dtf}
  
 
p = get_gridcols(df, out=False)
p
print ( p['code'] ) 
 

In [ ]:
sql = get_gridcols(df)
print( sql['code'])

In [ ]:
def get_sql(df,out=False):
    parms=[]
    for i,r in df.iterrows():
        s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()}({r['CHARACTER_MAXIMUM_LENGTH']}) = NULL"
        if r['DATA_TYPE'].upper() == 'INT':
            s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()} = NULL"  
        parms.append(f',{s}')

    into=', '.join([c for c in df['COLUMN_NAME'] if 'PK_' not in c])
    vals=', @'.join([c for c in df['COLUMN_NAME'] if 'PK_' not in c]) 
    ins=f'\n INSERT INTO @TABLE ({into}) \n VALUES (@{vals})'

    parms='\n'.join(parms)[:] 
    update=', '.join([f'{c}=@{c}' for c in df['COLUMN_NAME'] if 'PK_' not in c ])  
    ext=' AND ' + ' AND '.join([f'{c}=@{c}' for c in df['COLUMN_NAME'] if 'PK_' not in c ])

    sel=[] 
    exp=[]
    for i,r in df.iterrows():   
        s=f"\n, {r['COLUMN_NAME']} "
        if r['DATA_TYPE'].upper() == 'INT':   
            s=s+f"\n, (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist={r['COLUMN_NAME']}) AS [{r['COLUMN_NAME']}_Display]" 
            exp.append(f"\n, (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist={r['COLUMN_NAME']}) AS [{r['HEADER']}]")
        elif r['DATA_TYPE'].upper() == 'DATETIME':
            s=s+f"\n, FORMAT({r['COLUMN_NAME']}, 'MM/dd/yyyy') AS [{r['COLUMN_NAME']}_Display] " 
            exp.append(f"\n, FORMAT({r['COLUMN_NAME']}, 'MM/dd/yyyy') AS [{r['HEADER']}]") 
        else:
            exp.append(f"\n, {r['COLUMN_NAME']} AS [{r['HEADER']}]")
        sel.append(''+s)
    sel=''.join(sel)  
    exp=''.join(exp)  
    return {'parms':parms,'ins':ins, 'update':update, 'sel':sel, 'ext':ext, 'exp':exp  }


# df = sql_todf("SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='FEDRAMP_UPLOAD'", ctx.connstr) 
# df = normalizedf(df) 
#  df.columns
d=get_sql(df, out=False)
print( d['ins'].replace(',','\n,')  )